# Data preprocessing 資料前處理

## Cleaning numerical literals 將資料夾中numerical literals清理

In [1]:
import pandas as pd
import numpy as np

In [ ]:
path = "LitWD48K/"


# From LiterallyWikidata read data and check data

In [2]:
column_name = ["e","a","v"]
df48_ori = pd.read_csv(f"/projekte/tcl/tclext/kgc_chu/LiterallyWikidata/LitWD48K/numeric_literals.txt",sep="\t",names=column_name)

In [47]:
df48_ori[:5]

,e,a,v,xml,new_v
0,Q1000056,P1082_Q199,11032.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+110
1,Q1000056,P2044_Q11573,472.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+472
2,Q1000056,P2046_Q25343,45630000.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+456
3,Q1000138,P1082_Q199,1375.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+137
4,Q1000138,P2044_Q11573,1.0,^<http://www.w3.org/2001/XMLSchema#decimal>,+1


In [3]:
df48 = df48_ori

In [4]:
#取V值
df48[['v','xml']] = df48['v'].str.split('^', 1, expand=True)

In [5]:
# check the number of data
print(f'triple number of df48: {len(df48)}, num of df48_entity: {len(df48["e"].unique())}, num of df48_attribute: {len(df48["a"].unique())}')

triple number of df48: 324418, num of df48_entity: 47998, num of df48_attribute: 291


In [6]:
df48.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324418 entries, 0 to 324417
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   e       324418 non-null  object
 1   a       324418 non-null  object
 2   v       324418 non-null  object
 3   xml     324418 non-null  object
dtypes: object(4)
memory usage: 9.9+ MB


# Discovering some issues within data

## 3 value types

In [ ]:
#有三種數值型態
df48['xml'].value_counts()

## Time (year-month-date)

In [ ]:
# value type: time
df48[df48['xml']=='^<http://www.w3.org/2001/XMLSchema#dateTime>'][:5]

## Double: coordinate location

In [ ]:
# value type: double (geographic)
# coordinate location in original data is mixed. P625，P1335...(coordinate location)有經緯度
df48[df48["a"].str.contains("P1335")][:5]

## Decimal

In [ ]:
# value type: decimal
# same entity has multiple values 可能同entity在P1087有多個值
df48[df48["a"].str.contains("P1087_Q199")][:5]

In [ ]:
# P2299 has 2 units 有兩種QXXX
df48[df48["a"].str.contains("P2299")].a.unique()

In [ ]:
# some are meanful zero value but some are not
df_vzero=df48[df48["v"]=='0'].sort_values(by='a')

In [ ]:
df_vzero[:5]

In [ ]:
print(f"there are total {len(df_vzero)} data which has 0.0 in the value column")
print(f"there are total {len(df_vzero['a'].unique())} data which has 0.0 in the value column")

In [ ]:
df48[df48["a"].str.contains("P1082")].v

# Dealing with issues 處理以上問題: 
DataTime dtype: keep year data 時間單位 西元年 \\
Remove dulplicated data (same e,a but diff v)
Turn into value dtype to float 
所有變數的值成為可計算的值(有意義可讀取的值) \\
Rescale the values within the same attributes 同attribute value rescale \\
Drop out unreasonal 0 value 去0 \\
Drop out the unit (QXXX)去單位 \\
Keep Attribyes at least number of 15 data 每個屬性最少有15筆資料 \\


## value type: time

In [7]:
# 如果"v"是時間，取前四位數年代部分，np.where(condition, x,y)在condition下，把x取代y
df48["new_v"] = df48["v"].map(lambda x: x[:4])
df48.loc[:,"v"] = np.where((df48["xml"].str.contains("dateTime")), df48["new_v"], df48["v"])

In [8]:
df48["v"]=df48["v"].astype('float32')

## value type: double

In [ ]:
df48

In [ ]:
# remove double data
#neogeo = df48[~df48['xml'].str.contains('double')]

## value type: decimal

In [9]:
#取有100data的att #同時解決同p不同q
filter = df48.a.value_counts()
filter_index=filter.index[filter>=150]
df48 = df48[df48['a'].isin(filter_index)]

In [11]:
df48.a.value_counts()

P625_Longtiude    24952
P625_Latitude     24952
P2046_Q25343      24039
P1082_Q199        22299
P2044_Q11573      16734
                  ...  
P2219_Q199          164
P2131_Q4917         163
P4010_Q550207       156
P2299_Q550207       155
P1279_Q199          152
Name: a, Length: 93, dtype: int64

In [12]:
#處理a，去單位
df48[["pre_a",'Q']] = df48["a"].str.split('_', 1,expand=True)
df48.loc[:,"new_a"] = np.where((df48["a"].str.contains("Q")), df48["pre_a"], df48["a"])

/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [13]:
df48 = df48.loc[:,['e','new_a','v']]
df48.columns=['e','a','v']


In [14]:
#處理重複值 drop if [e,a] is the same
df48 = df48.drop_duplicates(subset=['e','a'],keep='last')

In [ ]:
#如果要取平均值:處理重複資料
# df48 = df48.groupby(['e', 'a'], as_index=False).mean()

In [16]:
df48.a.value_counts()

P625_Longtiude    24952
P625_Latitude     24952
P2046             21751
P1082             21446
P2044             16418
                  ...  
P1087               135
P8477               110
P8476               110
P3864                53
P1603                 8
Name: a, Length: 93, dtype: int64

In [17]:
filter = df48.a.value_counts()
filter_index=filter.index[filter>=150]
df48 = df48[df48['a'].isin(filter_index)]

In [ ]:
#處理0值
df48[df48['v']==0.0]

## zero value

In [20]:
# preparing idmap for checking real name
entity_idmap = {}
with open('Entities/entity_labels_en.txt',encoding="utf-8") as f:
    for line in f:
        k, v = line.strip().split('\t')
        entity_idmap[k.strip()] = v.strip()
attri_idmap={}
with open('Attributes/attribute_labels_en.txt',encoding="utf-8") as f:
    for line in f:
        k, v = line.strip().split('\t')
        attri_idmap[k.strip()] = v.strip()
attri_idmap['P625_Latitude']='coordinate location(latitude)'
attri_idmap['P625_Longtiude']='coordinate location(logtitude)'
attri_idmap['P1332_Longtiude']='coordinates of northernmost point'
attri_idmap['P1333_Longtiude']='coordinates of southernmost point'
attri_idmap['P1334_Longtiude']='coordinates of easternmost point'
attri_idmap['P1335_Longtiude']='coordinates of westernmost point'
df48.loc[:,'name_e']=df48['e'].map(entity_idmap)
df48.loc[:,'name_a']=df48['a'].map(attri_idmap)

/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [22]:
# 刪掉特定列，有11個不合理的0值變數
df48 = df48.drop(df48[(df48['a']=='P6509') & (df48['v']==0.0)].index)#'total goals in career'
df48 = df48.drop(df48[(df48['a']=='P6544') & (df48['v']==0.0)].index)#total points in career 
df48 = df48.drop(df48[(df48['a']=='P6543') & (df48['v']==0.0)].index)#total shots in career
df48 = df48.drop(df48[(df48['a']=='P6546') & (df48['v']==0.0)].index)#penalty minutes in career
df48 = df48.drop(df48[(df48['a']=='P6545') & (df48['v']==0.0)].index)#total assists in career 
df48 = df48.drop(df48[(df48['a']=='P1100') & (df48['v']==0.0)].index)#attendance 
df48 = df48.drop(df48[(df48['a']=='P1279') & (df48['v']==0.0)].index)##inflation rate
df48 = df48.drop(df48[(df48['a']=='P2121') & (df48['v']==0.0)].index)#prize money
df48 = df48.drop(df48[(df48['a']=='P3872') & (df48['v']==0.0)].index)#惠顧人數patronage

In [23]:
df48.reset_index(inplace=True)


In [25]:
df48= df48.drop('index',axis=1)
df48.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296457 entries, 0 to 296456
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   e       296457 non-null  object 
 1   a       296457 non-null  object 
 2   v       296457 non-null  float32
 3   name_e  296457 non-null  object 
 4   name_a  262436 non-null  object 
dtypes: float32(1), object(4)
memory usage: 10.2+ MB


In [ ]:
# 縮小domain

In [ ]:
filter = df48.a.value_counts()
filter_index=filter.index[filter>=150]
df48 = df48[df48['a'].isin(filter_index)]

In [26]:
print(f'triple number of df48: {len(df48)}, num of df48_entity: {len(df48["e"].unique())}, num of df48_attribute: {len(df48["a"].unique())}')

triple number of df48: 296457, num of df48_entity: 47939, num of df48_attribute: 87


In [ ]:
ent_typemap = {}
with open('Entities/entity_types.txt',encoding="utf-8") as f:
    for line in f:
        k, v = line.strip().split('\t')
        ent_typemap[k.strip()] = v.strip()

In [ ]:
df48['ent_type'] = df48['e'].map(ent_typemap)

In [ ]:
df48 = df48.loc[:,["e","a","v","name_e","name_a","ent_type"]]

In [29]:
print(f'triple number of df48: {len(df48)}, num of df48_entity: {len(df48["e"].unique())}, num of df48_attribute: {len(df48["a"].unique())}')

triple number of df48: 296457, num of df48_entity: 47939, num of df48_attribute: 87


In [46]:
df48.name_a.str.lower()

0                                population
1                 elevation above sea level
2                                      area
3                                population
4                 elevation above sea level
                        ...                
296452    coordinates of southernmost point
296453     coordinates of easternmost point
296454     coordinates of westernmost point
296455       coordinate location(logtitude)
296456       coordinate location(logtitude)
Name: name_a, Length: 296457, dtype: object

In [31]:
df48_p=df48

In [42]:
row_idx = []
def gather_data(name_att,df):
    df = df.reset_index(drop=True)
    ents = list(df[df['name_a']==name_att].e.unique())
    for ent in ents:
        list_ent=list(df[df['e']==ent].index)
        row_idx.extend(list_ent)
    return row_idx

In [45]:
ent_idx=gather_data('population',df48_p)

In [52]:
people_related_att = df48.iloc[list(set(ent_idx)),:]

In [54]:
people_related_att.to_csv('files_needed/pop_related.txt',index=False)

# type specific

In [ ]:
row_idx = []
def gather_data(name_att,df):
    df = df.reset_index(drop=True)
    a_type = list(df[df['name_a'].str.contains(name_att)].ent_type.unique())
    for type in a_type:
        list_type=list(df[df['ent_type']==type].index)
        row_idx.extend(list_type)
    return row_idx

In [ ]:
idx= gather_data("population",df48)
pop_related_att = df48.iloc[idx,:]
pop_related_att.reset_index(inplace=True)
pop_related_att=pop_related_att.drop('index',axis=1)

In [ ]:
pop_related_att.to_csv('files_needed/pop_related_att',index=False)

In [ ]:
df48[df48.name_a.str.contains("work")]

In [ ]:
idx2= gather_data("date of birth",df48)
people_related_att = df48.iloc[idx2,:]


In [ ]:
var_name = ["population","GDP (PPP)","PPP GDP per capita",
      "date of birth","date of death", "life expectancy",
      "total revenue","net profit",
      "area",
      "retirement age","age of majority","work period (start)","work period (end)"]

In [ ]:
people_related_att.name_a.value_counts()

In [ ]:
idx_all = []
for var in var_name:
    idx_all = gather_data(var,df48)


In [ ]:
idx_all

In [ ]:
df48.a.value_counts()

In [ ]:
list(set(idx_all))

In [ ]:
var_extracted = df48.iloc[list(set(idx_all)),:]

In [ ]:
var_extracted = var_extracted.reset_index(drop=True)

In [ ]:
df48.to_csv('/projekte/tcl/tclext/kgc/LiterallyWikidata/files_needed/nogeo_df48',index=False)
#var_extracted.to_csv('/projekte/tcl/tclext/kgc/LiterallyWikidata/files_needed/nogeo_df48_ver02',index=False)
.to_csv('/projekte/tcl/tclext/kgc/LiterallyWikidata/files_needed/nogeo_df48_ver02',index=False)

In [ ]:
var_extracted.info()

## Rescale decimal values

In [ ]:
#標準常態係數
from sklearn.preprocessing import StandardScaler
def attvalue2scale(df):
    scaler = StandardScaler()
    alist=list(df['a'].unique())
    for a_name in alist:
        attdf = df[df["a"]==a_name]
        scaler.fit(attdf.v.to_numpy().reshape(-1,1))
        result = scaler.transform(df.v.to_numpy().reshape(-1,1))
        return result


In [ ]:
x = attvalue2scale(df48)

In [ ]:
x

In [ ]:
df48['std_v']=x

In [ ]:
df48.a.value_counts()[:10]

In [ ]:
df48[df48.std_v==6.287763e+22]

In [ ]:
maxoutlier= df48[df48['a']=='P2046'].v > df48[df48['a']=='P2046'].v.mean() + 2 * df48[df48['a']=='P2046'].v.std()                          

In [ ]:
df48.iloc[maxoutlier.index,:]

In [ ]:
df48.loc[125347,'v']=4.239700e+8

In [ ]:
#minmax method
def attvalue2scale(df):
  alist=list(df['a'].unique())
  for a_name in alist:
    attdf = df[df["a"]==a_name]
    for i in range(len(attdf)):
      result = (df.v - attdf.v.min())/ (attdf.v.max()- attdf.v.min())
      return result


In [ ]:
df48

## Save the cleaning data 存檔

In [ ]:
df48 = df48.loc[:,['e','a','v','name_e','name_a','ent_type','std_v']]
df48.to_csv('/projekte/tcl/tclext/kgc/LiterallyWikidata/files_needed/nogeo_a89',index=False)


In [ ]:
#outlier not finish

## Split into train, valid and test sets


In [ ]:
from sklearn.model_selection import train_test_split
train_attri_data, valid_attri_data = train_test_split(df48, test_size=0.2,stratify=df48['a'])
#train_attri_data, valid_attri_data = train_test_split(df, test_size=0.2)
valid_attri_data, test_attri_data = train_test_split(valid_attri_data, test_size=0.5,stratify=valid_attri_data['a'])
#valid_attri_data, test_attri_data = train_test_split(valid_attri_data, test_size=0.5)

In [ ]:
print(f'training triples of df48: {len(train_attri_data)}, valid triples of df48: {len(valid_attri_data)}, test triples of df48: {len(test_attri_data)}')

In [ ]:
len(df48)

In [ ]:
valid_attri_data

In [ ]:
test_attri_data.a.value_counts()

In [ ]:
train_attri_data.a.value_counts()

In [ ]:
train_attri_data.to_csv(f'train_attri_data.csv',index=False)
valid_attri_data.to_csv(f'valid_attri_data.csv',index=False)
test_attri_data.to_csv(f'test_attri_data.csv',index=False)

In [ ]:
df_train_att= pd.read_csv(f'train_attri_data.csv')

In [ ]:
df_train_att['e'].unique()

In [ ]:
df_train_rel= pd.read_csv(f'train.txt',sep='\t',names=['s','p','o'])
df_test_rel= pd.read_csv(f'test.txt',sep='\t',names=['s','p','o'])
df_valid_rel= pd.read_csv(f'valid.txt',sep='\t',names=['s','p','o'])

In [ ]:
df_rel_all = pd.concat([df_train_rel,df_valid_rel,df_test_rel],axis=0)

In [ ]:
#e_train_list = list(df_train_rel.s.unique())
e_valid_list = list(df_valid_rel.s.unique())
e_test_list = list(df_test_rel.s.unique())

In [ ]:
valid_attri_data[valid_attri_data.e.isin(e_valid_list)==True]

In [ ]:
## all data index {ent:idx,rel:idx,att:idx}
entities = pd.read_csv('', sep='\t', names=['label', 'name'])
relations = pd.read_csv( 'Relations/relation_labels_en.txt', sep='\t', names=['label', 'name'])
attributes = attri_data.a.value_counts().index
dict_ent_2_idx = dict(zip(entities['label'], np.arange(0, len(entities), 1)))
dict_rel_2_idx = dict(zip(relations['label'], np.arange(0, len(relations), 1)))
dict_att_2_idx = dict(zip(attributes, np.arange(0, len(attributes), 1)))

## Dict contains all Graph objects
dict_all_2_idx = {}
dict_all_2_idx.update(self.dict_ent_2_idx)
dict_all_2_idx.update(self.dict_rel_2_idx)
dict_all_2_idx.update(self.dict_att_2_idx)



In [ ]:
dict_e2rv = dict()
for el in attri_data.values:
    #r = self.dict_att_2_idx[el[1]]
    attri = dict_att_2_idx[el[1]]
    v = round(el[2],5)
    e = dict_ent_2_idx[el[0]]
    if e in dict_e2rv:
        l = dict_e2rv[e]
        l.append([attri,v])
        dict_e2rv[e] = l
    else:
        dict_e2rv[e] = [[attri,v]]

In [ ]:
df = pd.read_csv('files_needed/nogeo_df48_var')

In [ ]:
filter
df.name_a.value_counts()